# Claim Review Search API interactions

In [99]:
import json
import requests
import csv

In [120]:
request_string = "https://factchecktools.googleapis.com/v1alpha1/claims:search"
sources = {"fullfact.org", "factcheck.afp.com", "factual.afp.com", "bbc.com", "boatos.org", "correctiv.org"}
sources_with_rating = {"bbc.com", "factcheck.afp.com", "boatos.org", }

In [64]:
def collectSourceClaims(publisher, query = "", page_size = 2):
    params = {'pageSize': page_size, 'query': query, 'reviewPublisherSiteFilter': publisher, 'key': api_key}
    response = requests.get(request_string, params)
    return response.json()

In [123]:
def processSource(publisher):
    claims = collectSourceClaims(publisher, query = "", page_size = 5000)
    file = "/home/atagarev/workspaces/WEV/wev-model/bootstrapping/results/source_" + publisher + ".csv"
    with open(file, mode='w') as loc_file:
        writer = csv.writer(loc_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        writer.writerow(['', 'claimReview_claimReviewed', 'rating_ratingValue',
                         'creativeWork_author_name', 'creativeWork_datePublished',
                         'claimReview_author_name', 'claimReview_datePublished',
                         'claimReview_url', 'extra_body'])
        i = 0
        for claim in claims['claims']:
            claimText = claim['text']
            claimant = claim['claimant'] if 'claimant' in claim else ""
            claimDate = claim['claimDate'] if 'claimDate' in claim else ""
            claimReview = claim['claimReview'][0]
            reviewer = claimReview['publisher']['name']
            reviewerSite = claimReview['publisher']['site']
            reviewUrl = claimReview['url']
            reviewText = claimReview['title']
            if not publisher in sources_with_rating:
                reviewText = claimReview['title'] + "\n\n" + claimReview['textualRating']
            reviewDate = claimReview['reviewDate']
            reviewRating = claimReview['textualRating'] if publisher in sources_with_rating else ""
            reviewLang = claimReview['languageCode']
            i += 1
            writer.writerow([i, claimText, reviewRating, claimant, claimDate, reviewer, reviewDate, reviewUrl, reviewText])

In [124]:
for source in sources:
    processSource(source)